In [7]:
import pyomo.environ as pe
import pyomo.opt as po
import math

# Data (same as Part 1)
factories = ["IJmuiden", "Segal", "South Wales"]
rebars = {"A": {"length": 2.4}, "B": {"length": 3.6}, "C": {"length": 4.2}}
longbars = {1: {"length": 9}, 2: {"length": 12}}
diameter = 0.057  # in meters
density = 7.85  # tons/m^3
production_capacity = {"IJmuiden": 12, "Segal": 10, "South Wales": 28}
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]
periods = [1, 2, 3, 4]

demand = {
    "A": {
        "Bochum": [2, 6, 5, 3],
        "Boenen": [4, 8, 5, 10],
        "Dortmund": [2, 7, 6, 5],
        "Gelsenkirchen": [5, 5, 5, 5],
        "Hagen": [19, 23, 25, 16],
        "Iserlohn": [13, 19, 17, 14],
        "Neuss": [20, 16, 14, 26],
        "Schwerte": [4, 5, 3, 4]
    },
    "B": {
        "Bochum": [4, 5, 7, 8],
        "Boenen": [5, 8, 12, 13],
        "Dortmund": [4, 5, 8, 10],
        "Gelsenkirchen": [9, 10, 6, 6],
        "Hagen": [15, 33, 31, 33],
        "Iserlohn": [22, 26, 20, 27],
        "Neuss": [12, 23, 30, 30],
        "Schwerte": [2, 8, 2, 6]
    },
    "C": {
        "Bochum": [6, 7, 7, 7],
        "Boenen": [6, 10, 15, 12],
        "Dortmund": [7, 6, 4, 12],
        "Gelsenkirchen": [10, 9, 9, 10],
        "Hagen": [12, 35, 33, 38],
        "Iserlohn": [14, 25, 23, 24],
        "Neuss": [22, 32, 31, 31],
        "Schwerte": [5, 6, 7, 2]
    }
}

fixed_cost = {"IJmuiden": 130, "Segal": 150, "South Wales": 100}
v = 0.5  # variable cost €/km/tonne
distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# New data: Inventory capacities (from Table 7)
inventory_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5
}

# Precompute parameters
w_r = {r: rebars[r]["length"] for r in rebars}
w_l = {l: longbars[l]["length"] for l in longbars}
area = math.pi * (diameter / 2) ** 2
weight_r = {r: density * area * w_r[r] for r in rebars}

# Create the model
model = pe.ConcreteModel()

# Define sets
model.F = pe.Set(initialize=factories)
model.R = pe.Set(initialize=rebars.keys())
model.L = pe.Set(initialize=longbars.keys())
model.A = pe.Set(initialize=customers)
model.T = pe.Set(initialize=periods)

# Define variables
model.x = pe.Var(model.R, model.L, model.F, model.T, domain=pe.NonNegativeIntegers)
model.y = pe.Var(model.L, model.F, model.T, domain=pe.NonNegativeIntegers)
model.z = pe.Var(model.F, model.A, model.T, domain=pe.Binary)
model.q = pe.Var(model.R, model.F, model.T, domain=pe.NonNegativeIntegers)  # New: rebars shipped
model.s = pe.Var(model.R, model.A, model.T, domain=pe.NonNegativeReals)  # New: inventory

# Constraints using ConstraintList and for loops
# 1. Cutting constraints
model.cutting_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        for l in model.L:
            expression = sum(model.x[r, l, f, t] * w_r[r] for r in model.R) <= model.y[l, f, t] * w_l[l]
            model.cutting_cnstr.add(expression)

# 2. Production capacity
model.capacity_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        expression = sum(model.x[r, l, f, t] * weight_r[r] for r in model.R for l in model.L) <= production_capacity[f]
        model.capacity_cnstr.add(expression)

# 3. Single sourcing
model.single_sourcing_cnstr = pe.ConstraintList()
for a in model.A:
    for t in model.T:
        expression = sum(model.z[f, a, t] for f in model.F) == 1
        model.single_sourcing_cnstr.add(expression)

# 4. Rebars shipped from factory
model.shipment_cnstr = pe.ConstraintList()
for r in model.R:
    for f in model.F:
        for t in model.T:
            expression = sum(model.x[r, l, f, t] for l in model.L) == model.q[r, f, t]
            model.shipment_cnstr.add(expression)

# 5. Inventory balance constraints
model.inventory_balance_cnstr = pe.ConstraintList()
for r in model.R:
    for a in model.A:
        for t in model.T:
            if t == 1:
                # Starting inventory is zero
                expression = model.s[r, a, t] == sum(model.z[f, a, t] * model.q[r, f, t] for f in model.F) - demand[r][a][t-1]
            else:
                # Inventory evolves based on previous inventory, shipments, and demand
                expression = model.s[r, a, t] == model.s[r, a, t-1] + sum(model.z[f, a, t] * model.q[r, f, t] for f in model.F) - demand[r][a][t-1]
            model.inventory_balance_cnstr.add(expression)

# 6. Inventory capacity constraints
model.inventory_capacity_cnstr = pe.ConstraintList()
for a in model.A:
    for t in model.T:
        expression = sum(model.s[r, a, t] * weight_r[r] for r in model.R) <= inventory_capacity[a]
        model.inventory_capacity_cnstr.add(expression)

# Objective function
objExpr = sum(fixed_cost[f] * model.z[f, a, t] for f in model.F for a in model.A for t in model.T) + \
          sum(v * distance[a][f] * model.z[f, a, t] * sum(model.q[r, f, t] * weight_r[r] for r in model.R)
              for f in model.F for a in model.A for t in model.T)
model.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result = solver.solve(model, tee=True)

# Display results
print(f"\nSolver Status: {result.solver.status}")
print(f"Termination Condition: {result.solver.termination_condition}")
if result.solver.status == po.SolverStatus.ok and result.solver.termination_condition == po.TerminationCondition.optimal:
    print(f"Objective value (Total Cost): {pe.value(model.obj):.2f} €")
    print("\nFactory assignments (z[f,a,t] = 1):")
    for f in model.F:
        for a in model.A:
            for t in model.T:
                if pe.value(model.z[f, a, t]) > 0.5:
                    print(f"z[{f}, {a}, {t}] = 1")
    print("\nRebars shipped (q[r,f,t] > 0):")
    for r in model.R:
        for f in model.F:
            for t in model.T:
                if pe.value(model.q[r, f, t]) > 0:
                    print(f"q[{r}, {f}, {t}] = {pe.value(model.q[r, f, t]):.0f}")
    print("\nInventory levels (s[r,a,t] > 0):")
    for r in model.R:
        for a in model.A:
            for t in model.T:
                if pe.value(model.s[r, a, t]) > 0:
                    print(f"s[{r}, {a}, {t}] = {pe.value(model.s[r, a, t]):.2f}")
    print("\nRebars cut (x[r,l,f,t] > 0):")
    for r in model.R:
        for l in model.L:
            for f in model.F:
                for t in model.T:
                    if pe.value(model.x[r, l, f, t]) > 0:
                        print(f"x[{r}, {l}, {f}, {t}] = {pe.value(model.x[r, l, f, t]):.0f}")
    print("\nLong bars used (y[l,f,t] > 0):")
    for l in model.L:
        for f in model.F:
            for t in model.T:
                if pe.value(model.y[l, f, t]) > 0:
                    print(f"y[{l}, {f}, {t}] = {pe.value(model.y[l, f, t]):.0f}")
else:
    print("No optimal solution found.")

Set parameter Username
Set parameter LicenseID to value 2617608
Academic license - for non-commercial use only - expires 2026-02-03
Read LP format model from file C:\Users\User\AppData\Local\Temp\tmpngod6nd4.pyomo.lp
Reading time = 0.02 seconds
x1: 136 rows, 324 columns, 468 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 136 rows, 324 columns and 468 nonzeros
Model fingerprint: 0x73e01c3e
Model has 288 quadratic objective terms
Model has 96 quadratic constraints
Variable types: 96 continuous, 228 integer (96 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  QObjective range [7e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+